# Bigram matches in Elasticsearch

This exercise is about getting ordered and unordered bigram matches using Elasticsearch.

In [1]:
from elasticsearch import Elasticsearch
from pprint import pprint

import ipytest
import pytest

ipytest.autoconfig()

## Indexing a toy collection 

This time, we store **term position information** and perform minimal stemming, i.e., removing only plurals (for that, we specify a custom analyzer).

Check the [Elasticsearch documentation on analyzers](https://www.elastic.co/guide/en/elasticsearch/reference/current/analyzer.html).

In [2]:
INDEX_NAME = "toy_index"  

INDEX_SETTINGS = {
    'settings' : {
        'index' : {
            "number_of_shards" : 1,
            "number_of_replicas" : 1
        },
        'analysis': {
            'analyzer': {
                'my_english_analyzer': {
                    'type': "custom",
                    'tokenizer': "standard",
                    'stopwords': "_english_",
                    'filter': [
                        "lowercase",
                        "english_stop",
                        "filter_english_minimal"
                    ]                
                }
            },
            'filter' : {
                'filter_english_minimal' : {
                    'type': "stemmer",
                    'name': "minimal_english"
                },
                'english_stop': {
                    'type': "stop",
                    'stopwords': "_english_"
                }
            },
        }
    },
    'mappings': {
        'properties': {
            'title': {
                'type': "text",
                'term_vector': "with_positions",
                'analyzer': "my_english_analyzer"
            },
            'content': {
                'type': "text",
                'term_vector': "with_positions",
                'analyzer': "my_english_analyzer"
            }
        }
    }
}

In [3]:
DOCS = {
    1: {"title": "Rap God",
        "content": "gonna, gonna, Look, I was gonna go easy on you and not to hurt your feelings"
        },
    2: {"title": "Lose Yourself",
        "content": "Yo, if you could just, for one minute Or one split second in time, forget everything Everything that bothers you, or your problems Everything, and follow me"
        },
    3: {"title": "Love The Way You Lie",
        "content": "Just gonna stand there and watch me burn But that's alright, because I like the way it hurts"
        },
    4: {"title": "The Monster",
        "content": ["gonna gonna I'm friends with the monster", "That's under my bed Get along with the voices inside of my head"]
        },
    5: {"title": "Beautiful",
        "content": "Lately I've been hard to reach I've been too long on my own Everybody has a private world Where they can be alone"
        },
    6: {"title": "Fake Eminem 1",
        "content": "This is not real Eminem, just some text to get more matches for a split second for a split second."
        },
    7: {"title": "Fake Eminem 2",
        "content": "I have a monster friend and I'm friends with the monster and then there are some more friends who are monsters."
        },
}

In [4]:
es = Elasticsearch()

In [5]:
if es.indices.exists(INDEX_NAME):
    es.indices.delete(index=INDEX_NAME)
    
es.indices.create(index=INDEX_NAME, body=INDEX_SETTINGS)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'toy_index'}

Testing our analyzer.

In [6]:
es.indices.analyze(index=INDEX_NAME, body={'analyzer': 'my_english_analyzer', 'text': 'monsters in my bed'})

{'tokens': [{'token': 'monster',
   'start_offset': 0,
   'end_offset': 8,
   'type': '<ALPHANUM>',
   'position': 0},
  {'token': 'my',
   'start_offset': 12,
   'end_offset': 14,
   'type': '<ALPHANUM>',
   'position': 2},
  {'token': 'bed',
   'start_offset': 15,
   'end_offset': 18,
   'type': '<ALPHANUM>',
   'position': 3}]}

In [7]:
for doc_id, doc in DOCS.items():
    es.index(index=INDEX_NAME, id=doc_id, body=doc)

Notice that you also get term position information when requesting a term vector.

In [8]:
tv = es.termvectors(index=INDEX_NAME, id=2, fields='title,content')
pprint(tv)

{'_id': '2',
 '_index': 'toy_index',
 '_type': '_doc',
 '_version': 1,
 'found': True,
 'term_vectors': {'content': {'field_statistics': {'doc_count': 7,
                                                   'sum_doc_freq': 85,
                                                   'sum_ttf': 101},
                              'terms': {'bother': {'term_freq': 1,
                                                   'tokens': [{'position': 18}]},
                                        'could': {'term_freq': 1,
                                                  'tokens': [{'position': 3}]},
                                        'everything': {'term_freq': 3,
                                                       'tokens': [{'position': 15},
                                                                  {'position': 16},
                                                                  {'position': 23}]},
                                        'follow': {'term_freq': 1,
                    

## Recovering ordered sequence of terms from inverted index

This method returns the sequence of terms for a given document field, with `None` values for stopwords that got removed.

In [9]:
def get_term_sequence(es, doc_id, field):
    tv = es.termvectors(index=INDEX_NAME, id=doc_id, fields=[field])
    # We first put terms in a position-indexed dict.
    pos = {}
    for term, tinfo in tv['term_vectors'][field]['terms'].items():
        for token in tinfo['tokens']:
            pos[token['position']] = term
    # Then, turn that dict to a list.
    seq = [None] * (max(pos.keys()) + 1)
    for p, term in pos.items():
        seq[p] = term
    return seq

Tests.

In [10]:
%%run_pytest[clean]

def test_get_term_sequence():
    assert get_term_sequence(es, 4, 'title') == [None, 'monster']
    assert get_term_sequence(es, 7, 'content') == ['i', 'have', None, 'monster', 'friend', None, "i'm", 'friend', None, None, 'monster', None, None, None, None, 'some', 'more', 'friend', 'who', None, 'monster']

.                                                                                  [100%]
1 passed in 0.02s


## Getting ordered bigram matches

Use the `get_term_sequence()` method to get the document field's content as a sequence of terms, then check for ordered bigram matches yourself.

In [11]:
def count_ordered_bigram_matches(es, doc_id, field, bigram):
    """Counts the number of ordered bigram matches in a given document field. 
    
    Args:
        es: Elasticsearch instance
        doc_id: Document ID
        field: Document field
        bigram: A sequence of two terms given as a list
    
    Returns:
        Number of times the bigram can be found in this exact order.
    """
    # Get the document field's content as a sequence of terms.
    text = get_term_sequence(es, doc_id, field)
    # Count the number of matches    
    count = 0
    for i in range(len(text) - 1):
        if text[i] == bigram[0]:
            if text[i + 1] == bigram[1]:
                count += 1
    return count

Tests.

In [12]:
%%run_pytest[clean]

@pytest.mark.parametrize('doc_id, field, bigram, correct_value', [
    (6, 'content', ['split', 'second'], 2),
    (2, 'content', ['split', 'second'], 1),
    (1, 'content', ['split', 'second'], 0),
])
def test_count_ordered_bigram_matches(doc_id, field, bigram, correct_value):
    assert count_ordered_bigram_matches(es, doc_id, field, bigram) == correct_value

...                                                                                [100%]
3 passed in 0.03s


## Getting unordered bigram matches

As before, use the `get_term_sequence()` method to get the document field's content as a sequence of terms, then check for ordered bigram matches yourself.

In [13]:
def count_unordered_bigram_matches(es, doc_id, field, bigram, w=4):
    """Counts the number of unordered bigram matches in a given document field. 
    
    Args:
        es: Elasticsearch instance
        doc_id: Document ID
        field: Document field
        bigram: A sequence of two terms given as a list
        w: The maximum distance between the two query terms that still counts as a match
    
    Returns:
        Number of times the bigram can be found within a distance of w from each other in any order.
    """
    text = get_term_sequence(es, doc_id, "content")
    count = 0
    for i in range(len(text) - 1):
        if text[i] in bigram:
            other_term = bigram[0] if text[i] == bigram[1] else bigram[1]
            if other_term in text[i+1:i+w]:
                count += 1
    return count

Tests.

In [14]:
%%run_pytest[clean]

@pytest.mark.parametrize('doc_id, field, bigram, correct_value', [
    (7, 'title', ['friend', 'monster'], 3),
    (4, 'title', ['friend', 'monster'], 1),
    (1, 'title', ['friend', 'monster'], 0),
])
def test_count_ordered_bigram_matches(doc_id, field, bigram, correct_value):
    assert count_unordered_bigram_matches(es, doc_id, field, bigram) == correct_value

...                                                                                [100%]
3 passed in 0.02s


## Feedback

Please give (anonymous) feedback on this exercise by filling out [this form](https://forms.gle/2jPayczbFhEcC9K68).